In [1131]:
#STREAMS2TABLE
#written by Zachary J.J. Roper
#z.roper@vanderbilt.edu, zach@roper.io

#A script for converting raw event codes and timestamps into a trial-item dataset

In [1132]:
#This notebook requires the following modules
import os
import numpy
import csv

In [1133]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [1134]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [1135]:
events.count('2756') #catch trials

3

In [1136]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [1137]:
#uberlist

In [1138]:
'2810' in uberlist[74][2]

False

In [1139]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [1140]:
#RT

In [1141]:
#The following provides indexed lists for each of the individual event codes of interest 

In [1142]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [1143]:
#float(times[74])-float(times[70])

In [1144]:
#Saccade

In [1145]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [1146]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [1147]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [1148]:
#print(TimeTrials[0])

In [1149]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [1150]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
tloc = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0
    #if '5000' in trial:
    #    tloc[i] = 360
    if '5045' in trial:
        tloc[i] = 45
    elif '5090' in trial:
        tloc[i] = 90
    elif '5135' in trial:
        tloc[i] = 135
    elif '5180' in trial:
        tloc[i] = 180
    elif '5225' in trial:
        tloc[i] = 225
    elif '5270' in trial:
        tloc[i] = 270
    elif '5315' in trial:
        tloc[i] = 315
    else:
        tloc[i] = 0

In [1151]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [1152]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [1153]:
#RT

In [1154]:
checksum = sum(error)+sum(abort)+sum(catch)

In [1155]:
checksum

674

In [1156]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i],tloc[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT', 'TarLoc']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT,tloc))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 0 0 1 0 0 1799.9873000000002 270
2 1 0 0 0 0 0 225
3 1 0 0 0 0 0 0
4 0 1 0 1 0 169.98379999999997 0
5 0 1 0 1 0 171.95010000000002 180
6 1 0 0 0 0 0 45
7 0 1 0 1 0 176.0051000000003 225
8 0 1 0 1 0 199.96669999999722 45
9 0 1 0 1 0 146.02230000000054 0
10 1 0 0 0 0 0 90
11 1 0 0 0 0 0 135
12 0 0 0 1 0 159.98980000000302 180
13 0 0 0 1 0 203.98059999999896 135
14 1 0 0 0 0 0 90
15 1 0 0 0 0 0 270
16 0 1 0 1 0 209.96089999999822 270
17 0 1 0 1 0 224.0103999999992 90
18 0 1 0 1 0 177.9713999999949 0
19 0 1 0 1 0 166.01090000000113 0
20 0 1 0 1 0 163.96280000000115 0
21 0 1 0 1 0 173.95709999999963 135
22 1 0 0 0 0 0 0
23 1 0 0 0 0 0 135
24 0 1 0 1 0 179.97819999999774 180
25 0 1 0 1 0 101.99020000000019 90
26 1 0 0 0 0 0 270
27 1 0 0 0 0 0 270
28 0 1 0 1 0 207.95390000000043 90
29 0 1 0 1 0 156.01650000000518 180
30 1 0 0 0 0 0 180
31 1 0 0 0 0 0 180
32 1 0 0 0 0 0 270
33 0 1 0 1 0 197.95969999999943 90
34 0 1 0 1 0 143.97430000000168 315
35 1 0 0 0 0 0 180
36 0 1 0 1 0 189.972599999993

720 1 0 0 0 0 0 225
721 0 1 0 1 0 207.99499999999534 270
722 0 1 0 1 0 277.99540000001434 135
723 0 1 0 1 0 229.99049999995623 225
724 0 1 0 1 0 198.00059999991208 45
725 1 0 0 0 0 0 135
726 1 0 0 0 0 0 180
727 1 0 0 0 0 0 225
728 0 0 0 1 0 199.96669999998994 45
729 0 1 0 1 0 179.97810000006575 180
730 1 0 0 0 0 0 180
731 1 0 0 0 0 0 315
732 1 0 0 0 0 0 225
733 1 0 0 0 0 0 270
734 0 0 1 0 0 204.02189999993425 225
735 0 0 1 0 0 217.98889999999665 315
736 0 1 0 1 0 216.0230000000447 225
737 0 1 0 1 0 231.95649999997113 225
738 0 0 0 1 0 212.00899999996182 315
739 0 1 0 1 0 193.94559999997728 90
740 1 0 0 0 0 0 180
741 1 0 0 0 0 0 0
742 0 1 0 1 0 173.95709999999963 45
743 0 1 0 1 0 176.0050999999512 135
744 1 0 0 0 0 0 45
745 1 0 0 0 0 0 180
746 1 0 0 0 0 0 90
747 0 1 0 1 0 183.99239999998827 270
748 0 0 1 0 0 239.98479999997653 45
749 1 0 0 0 0 0 270
750 1 0 0 0 0 0 225
751 1 0 0 0 0 0 135
752 1 0 0 0 0 0 45
753 1 0 0 0 0 0 315
754 1 0 0 0 0 0 45
755 1 0 0 0 0 0 225
756 1 0 0 0 0 0 45
75

In [1157]:
sum(catch)

3